<a href="https://colab.research.google.com/github/fikriansyahf/Machine-Learning-Lesson-Telkom-University/blob/main/Week_14_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install --upgrade pip

     |████████████████████████████████| 1.7 MB 14.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [9]:
pip install tensorflow

In [27]:
pip install --upgrade keras

**Sequence to Sequence Model**

In [30]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y
 
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y
 
# define LSTM
n_in = 5
n_out = 5
encoded_length = 100
batch_size = 7
model = Sequential()
model.add(LSTM(20, batch_input_shape=(batch_size, n_in, encoded_length), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(500):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

3/3 - 2s - loss: 4.5993 - accuracy: 0.0571 - 2s/epoch - 670ms/step
3/3 - 0s - loss: 4.6072 - accuracy: 0.0000e+00 - 10ms/epoch - 3ms/step
3/3 - 0s - loss: 4.6049 - accuracy: 0.0190 - 10ms/epoch - 3ms/step
3/3 - 0s - loss: 4.6097 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6039 - accuracy: 0.0381 - 12ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6038 - accuracy: 0.0286 - 10ms/epoch - 3ms/step
3/3 - 0s - loss: 4.6016 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6037 - accuracy: 0.0190 - 10ms/epoch - 3ms/step
3/3 - 0s - loss: 4.6026 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6030 - accuracy: 0.0095 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.6026 - accuracy: 0.0000e+00 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5976 - accuracy: 0.0000e+00 - 10ms/epoch - 3ms/step
3/3 - 0s - loss: 4.5949 - accuracy: 0.0190 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5978 - accuracy: 0.0095 - 11ms/epoch - 4ms/step
3/3 - 0s - loss: 4.5928 - accuracy: 0.

**Encoder-Decoder Model**

In [31]:
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
 
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
	return [randint(0, 99) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]
 
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(sequence)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = sequence.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y
 
# prepare data for the LSTM
def get_data(n_in, n_out):
	# generate random sequence
	sequence = generate_sequence()
	# one hot encode
	encoded = one_hot_encode(sequence)
	# convert to X,y pairs
	X,y = to_supervised(encoded, n_in, n_out)
	return X,y
 
# define LSTM
n_in = 5
n_out = 2
encoded_length = 100
batch_size = 21
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
model.add(RepeatVector(n_out))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_data(n_in, n_out)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
# evaluate LSTM
X,y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
# decode all pairs
for i in range(len(X)):
	print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

Streaming output truncated to the last 5000 lines.
1/1 - 0s - loss: 4.6029 - accuracy: 0.0714 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6049 - accuracy: 0.0000e+00 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6013 - accuracy: 0.0000e+00 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6008 - accuracy: 0.0000e+00 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.5988 - accuracy: 0.0476 - 19ms/epoch - 19ms/step
1/1 - 0s - loss: 4.6003 - accuracy: 0.1190 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6004 - accuracy: 0.0238 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6009 - accuracy: 0.0238 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.5993 - accuracy: 0.0238 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.6012 - accuracy: 0.0000e+00 - 12ms/epoch - 12ms/step
1/1 - 0s - loss: 4.5977 - accuracy: 0.0238 - 18ms/epoch - 18ms/step
1/1 - 0s - loss: 4.5970 - accuracy: 0.0238 - 13ms/epoch - 13ms/step
1/1 - 0s - loss: 4.5940 - accuracy: 0.0000e+00 - 14ms/epoch - 14ms/step
1/1 - 0s - loss: 4.5995 - accuracy: 0.0952 - 